In [1]:
import os
import base64

from dotenv import load_dotenv
from openai import OpenAI
from io import BytesIO
from PIL import Image

In [2]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
print(openai_api_key[:7])

sk-proj


In [3]:
openai = OpenAI()

In [4]:
def artist(city):
    response = openai.images.generate(
        model="dall-e-3",
        prompt=f"Сгенерируй забавную картинку под шутку: Звери умнее, чем кажутся, особенно когда дело касается поиска еды в мусорном баке",
        size="1024x1024",
        n=1,
        response_format="b64_json"
    )

    image_base64 = response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [ ]:
image = artist("Asterdam")
# image.show()
display(image)

In [5]:
from pydub import AudioSegment
from pydub.playback import play

In [6]:
def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="alloy",
        input=message
    )

    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    play(audio)

In [7]:
talker("Hi there, you are awesome!")

Input #0, wav, from '/var/folders/ny/bjrptr9d4f9c3mmxj10zzcl40000gp/T/tmpxug_2fvx.wav':
  Duration: 00:00:01.56, bitrate: 384 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 24000 Hz, 1 channels, s16, 384 kb/s
   1.51 M-A:  0.000 fd=   0 aq=    0KB vq=    0KB sq=    0B 

In [8]:
system_prompt = "You are a helpful assistant for an Airline called FlightAI. "
system_prompt += "Give short, courteous answers, no more than 1 sentence. "
system_prompt += "Always be accurate. If you don't know the answer, say so."

In [9]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(city):
    print(f'Tool get_ticket_price is called for {city}')
    city = city.lower()
    return ticket_prices.get(city, "Unknown")


In [10]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            }
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

In [11]:
tools=[{"type": "function", "function": price_function}]

In [12]:
import json

In [13]:
def handle_tool_call(tool_call_msg):
    tool_call = tool_call_msg.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('city')
    price = get_ticket_price(city)

    response = {
        'role': 'tool',
        'content': json.dumps({'city': city, 'price': price}),
        'tool_call_id': tool_call.id
    }
    
    return response, city

In [14]:
def chat_old(message, history):
    messages = [{'role': 'system', 'content': system_prompt}] + history
    messages += [{'role': 'user', 'content': message}]
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools
    )

    if response.choices[0].finish_reason == 'tool_calls':
        tool_call_msg = response.choices[0].message
        tool_call_response, city = handle_tool_call(tool_call_msg)
        messages.append(tool_call_msg)
        messages.append(tool_call_response)
        response = openai.chat.completions.create(model='gpt-4o-mini', messages=messages)

    return response.choices[0].message.content
    

In [15]:
import gradio as gr

In [16]:
gr.ChatInterface(fn=chat_old, type='messages').launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [17]:
def chat(history):
    print(history)
    
    messages = [{'role': 'system', 'content': system_prompt}] + history
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=tools
    )
    image = None

    if response.choices[0].finish_reason == 'tool_calls':
        tool_call_msg = response.choices[0].message
        tool_call_response, city = handle_tool_call(tool_call_msg)
        messages.append(tool_call_msg)
        messages.append(tool_call_response)
        image = artist(city)
        response = openai.chat.completions.create(model='gpt-4o-mini', messages=messages)

    reply = response.choices[0].message.content
    history += [{'role': 'assistant', 'content': reply}]

    talker(reply)
    
    return history, image

In [18]:
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{'role': 'user', 'content': message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=[chatbot], outputs=[chatbot, image_output]
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
